In [1]:
%cd ..
import os
import time

/data/apperception/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

/home/local/apperception/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/apperception/weights/ultralytics_yolov5_master


Using cuda:0


YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
NUSCENES_PROCESSED_DATA_FULL = "NUSCENES_PROCESSED_DATA_FULL"
if NUSCENES_PROCESSED_DATA_FULL in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA_FULL]
else:
    DATA_DIR = "/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Trainval"
NUSCENES_RAW_DATA_FULL = "NUSCENES_RAW_DATA_FULL"
if NUSCENES_RAW_DATA_FULL in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA_FULL]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval"

In [4]:
import cv2
def construct_video(frames, size=(1600, 900), base=False, vid_name=None):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig15_"+vid_name if vid_name else f"./outputs/{'fig15_base' if base else 'fig15_optimized'}.avi"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [12]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

cam = camera
car1 = objects[0]
opposite_car = objects[1]
car2 = objects[2]

world = world.filter(
    (F.like(car1.type, 'car') | F.like(car1.type, 'truck')) &
    (F.like(car2.type, 'car') | F.like(car2.type, 'truck')) &
    (F.like(opposite_car.type, 'car') | F.like(opposite_car.type, 'truck')) &
    (opposite_car.id != car2.id) &
    (car1.id != car2.id) &
    (car1.id != opposite_car.id) &

    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
    (F.view_angle(car1.traj@cam.time, cam.ego) < 70 / 2) &
    (F.distance(cam.ego, car1.traj@cam.time) < 50) &
#     F.angle_between(F.facing_relative(car1.traj@cam.time, cam.ego), -15, 15) &
#     F.angle_between(F.facing_relative(car1.traj@cam.time, F.road_direction(car1.traj@cam.time, cam.ego)), -15, 15) &
    F.ahead(car1.traj@cam.time, cam.ego) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
    (F.convert_camera(opposite_car.traj@cam.time, cam.ego) > [-10, 0]) &
    (F.convert_camera(opposite_car.traj@cam.time, cam.ego) < [-1, 50]) &
    F.angle_between(F.facing_relative(opposite_car.traj@cam.time, cam.ego), 140, 180) &
    (F.distance(opposite_car@cam.time, car2@cam.time) < 40) &
#     F.angle_between(F.facing_relative(car2.traj@cam.time, F.road_direction(car2.traj@cam.time, cam.ego)), -15, 15) &
    F.ahead(car2.traj@cam.time, opposite_car.traj@cam.time)
)


In [6]:
base_query_start = time.time()
preprocess(world, DATA_DIR, ['scene-0207-CAM_FRONT'])
print(f'base query time: {time.time()-base_query_start}')

YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0207-CAM_FRONT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:03<00:00, 25.24it/s]


None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:02<00:00, 112.66it/s]


...

None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/apperception/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:09<00:00, 24.79it/s]


None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking3D.From2DAndRoad


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:00<00:00, 7003.61it/s]


None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  SegmentTrajectory.FromTracking3D
None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
obj_id 1
obj_id 2
obj_id 3
obj_id 4
obj_id 5
obj_id 6
obj_id 8
obj_id 9
obj_id 11
obj_id 17
obj_id 23
obj_id 25
obj_id 32
obj_id 37
obj_id 42
obj_id 44
obj_id 47
obj_id 49
obj_id 53
obj_id 54
obj_id 56
obj_id 70
obj_id 74
obj_id 77
obj_id 79
obj_id 80
obj_id 82
obj_id 84
obj_id 87
obj_id 91
obj_id 95
obj_id 96
obj_id 100
obj_id 102
obj_id 105
Time taken to insert trajectories: 0.18668484687805176
info found [[5, 791.0, 41

In [13]:
id_time_camId_filename_base = world.get_id_time_camId_filename(3)

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId,t1.itemId,t2.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId) JOIN (SELECT * FROM item_general_trajectory) as t2 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND (t2.objectType LIKE 'car' OR t2.objectType LIKE 'truck') AND (t1.objectType LIKE 'car' OR t1.objectType LIKE 'truck') AND (t1.itemId<>t2.itemId) AND (t0.itemId<>t2.itemId) AND (t0.itemId<>t1.itemId) AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (viewAngle(valueAtTimestamp(t0.trajCentroids,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.trajCentroids,timestamp))<50) AND ahead(valueAtTimestamp(t0.trajCe

In [14]:
id_time_camId_filename_base

[('scene-0207-CAM_FRONT_obj_5',
  'scene-0207-CAM_FRONT_obj_17',
  'scene-0207-CAM_FRONT_obj_32',
  datetime.datetime(2018, 7, 26, 9, 18, 39, 262404, tzinfo=datetime.timezone.utc),
  'scene-0207-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621919262404.jpg'),
 ('scene-0207-CAM_FRONT_obj_5',
  'scene-0207-CAM_FRONT_obj_37',
  'scene-0207-CAM_FRONT_obj_17',
  datetime.datetime(2018, 7, 26, 9, 18, 40, 12404, tzinfo=datetime.timezone.utc),
  'scene-0207-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920012404.jpg'),
 ('scene-0207-CAM_FRONT_obj_5',
  'scene-0207-CAM_FRONT_obj_37',
  'scene-0207-CAM_FRONT_obj_17',
  datetime.datetime(2018, 7, 26, 9, 18, 40, 112404, tzinfo=datetime.timezone.utc),
  'scene-0207-CAM_FRONT',
  'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920112404.jpg'),
 ('scene-0207-CAM_FRONT_obj_5',
  'scene-0207-CAM_FRONT_obj_37',
  'scene-0207-CAM_FRONT_obj_17',
  datetime.datetime(2018

In [15]:
base_results = [e[5] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [16]:
preprocess(world, DATA_DIR, ['scene-0207-CAM_FRONT'], base=False)

YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0207-CAM_FRONT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:03<00:00, 21.23it/s]


None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:02<00:00, 114.29it/s]


...

number of skipped 1
{None: 221, 'ego_exit_segment': 6, 'car_exit_segment': 1}
total_run_time 10.059957504272461
total_detection_time 8.1523277759552
total_generate_sample_plan_time 1.8109626770019531
229
229
  filtered frames: 99.56331877729258%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/apperception/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:08<00:00, 26.48it/s]


None
229
  filtered frames: 99.56331877729258%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  Tracking3D.From2DAndRoad


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:00<00:00, 8059.88it/s]


None
229
  filtered frames: 99.56331877729258%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  SegmentTrajectory.FromTracking3D
None
229
  filtered frames: 99.56331877729258%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKK.
obj_id 1
obj_id 2
obj_id 3
obj_id 4
obj_id 5
obj_id 6
obj_id 8
obj_id 9
obj_id 11
obj_id 17
obj_id 23
obj_id 25
obj_id 28
obj_id 30
obj_id 34
obj_id 36
obj_id 40
obj_id 43
obj_id 44
obj_id 46
obj_id 57
obj_id 62
obj_id 63
obj_id 69
Time taken to insert trajectories: 0.18945074081420898
info found [[5, 791.0, 412.0, 101.0, 124.0, (f=72 o=0), datetime.datetime(2018, 7, 26, 9, 18, 40, 662404), None, N

In [17]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(3)
print(f'optimized query time: {time.time()-optimized_query_start}')


execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId,t1.itemId,t2.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId) JOIN (SELECT * FROM item_general_trajectory) as t2 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND (t2.objectType LIKE 'car' OR t2.objectType LIKE 'truck') AND (t1.objectType LIKE 'car' OR t1.objectType LIKE 'truck') AND (t1.itemId<>t2.itemId) AND (t0.itemId<>t2.itemId) AND (t0.itemId<>t1.itemId) AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (viewAngle(valueAtTimestamp(t0.trajCentroids,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.trajCentroids,timestamp))<50) AND ahead(valueAtTimestamp(t0.trajCe

In [18]:
optimized_results = [e[5] for e in id_time_camId_filename]
construct_video(optimized_results)

In [19]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

0


In [20]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")

0


In [21]:
investigation = [[5, 806.0, 414.0, 95.0, 121.0], [17, 186.0, 312.0, 395.0, 339.0], [37, 645.0, 475.0, 76.0, 63.0]]

In [22]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open('/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval/samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for obj in investigation:
    obj_id = obj[0]
    x,y,w,h = obj[1:5]
    plt.text(x, y+15, 'obj'+str(obj_id))
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()